In [2]:
#importing processed data
import pandas as pd
df = pd.read_csv('all_data.csv', index_col = 0)

/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
#dropping all features with NAN
df = df.dropna(axis=1,how='any')

In [34]:
#Checking whether application is in both 
df_check = None
frames = []
df_v = []
df_p = []
for name, item in df.groupby(['kernelname', 'input']):
    df_1 = df[ df['kernelname'] == name[0] ]
    df_2 = df_1[ df_1['input'] == name[1] ]
    if len(df_2) == 2:
        
        df_v = df_2[df_2['architecture'] == 'V100']
        df_p = df_2[df_2['architecture'] == 'P100']
        df_p['ipc'][0] = df_v.iloc[0]['ipc']
        frames.append(df_p)
     
new_df = pd.concat(frames)

#print(new_df)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


                              inst_per_warp kernelname  branch_efficiency  \
gaussianP100_N128_Fan1            18.438976       Fan1           0.999381   
gaussianP100_N160_Fan1            19.718160       Fan1           0.999487   
gaussianP100_N192_Fan1            21.008508       Fan1           0.999533   
gaussianP100_N224_Fan1            22.269339       Fan1           0.999646   
gaussianP100_N256_Fan1            23.574510       Fan1           0.999572   
gaussianP100_N288_Fan1            24.902221       Fan1           0.999416   
gaussianP100_N320_Fan1            26.126567       Fan1           0.999654   
gaussianP100_N352_Fan1            27.384081       Fan1           0.999786   
gaussianP100_N384_Fan1            33.129243       Fan1           0.999698   
gaussianP100_N416_Fan1            29.958283       Fan1           0.999756   
gaussianP100_N448_Fan1            31.239653       Fan1           0.999812   
gaussianP100_N480_Fan1            32.521007       Fan1           0.999797   

In [97]:
#Splitting data with respect to architure
df_P100 = df[ df['architecture'] == 'P100' ]
df_V100 = df[ df['architecture'] == 'V100' ]


In [111]:
#Normalize Data 
#Columns with text: architecture, kernel, input, 
#application_name, kernalname
# new_def has p100 app metrics with v100 ipc
# Normalize df_p100

In [112]:
#Dropping all columns with strings
df_p100_norm = df_P100.drop(columns=['shared_utilization','stall_other','single_precision_fu_utilization','architecture','kernel','input','application_name','kernelname','ipc'])

In [113]:
new_def_norm = new_df.drop(columns=['shared_utilization','stall_other','single_precision_fu_utilization','architecture','kernel','input','ipc','application_name','kernelname'])

## Normalized data for mixed data metrics

In [114]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

#ipc values
new_def_ipc = new_df['ipc']
new_def_ipc_values = new_def_ipc.values
#new_def_ipc_norm = MinMaxScaler().fit_transform(new_def_ipc_values)

#v100+p100 combined
new_def_norm_values = new_def_norm.values
new_def_norm = MinMaxScaler().fit_transform(new_def_norm_values)

#p100 only normalized
df_p100_norm = df_p100_norm.values
df_p100_norm = MinMaxScaler().fit_transform(df_p100_norm)
p100_ipc = df_P100['ipc']
p100_ipc_values = p100_ipc.values

## Transfer Learning Model

In [115]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


# Create your first MLP in Keras
from keras.models import Sequential
from keras.layers import Dense
import numpy
# fix random seed for reproducibility
numpy.random.seed(7)
# load pima indians dataset
#dataset = pd.read_csv("p100_only.csv", delimiter=",").values
# split into input (X) and output (Y) variables
X = df_p100_norm
Y = p100_ipc
#X = dataset[:,0:8]
#Y = dataset[:,8]
# create model
def my_model():
    model = Sequential()
    model.add(Dense(12, input_dim=113, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    # Fit the model
    ## model.fit(X, Y, epochs=10, batch_size=10) ##works

    """
    # evaluate the model
    scores = model.evaluate(X, Y)
    print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))"""
    return model

In [116]:
print(X.shape)
print(Y.shape)

(13866, 113)
(13866,)


In [117]:
model = my_model()
model.fit(X, Y, epochs=10, batch_size=10)

Epoch 1/10
13866/13866 [==============================] - 2s 127us/step - loss: 0.3830
Epoch 2/10
13866/13866 [==============================] - 1s 103us/step - loss: 0.3483
Epoch 3/10
13866/13866 [==============================] - 1s 105us/step - loss: 0.3480
Epoch 4/10
13866/13866 [==============================] - 1s 102us/step - loss: 0.3479
Epoch 5/10
13866/13866 [==============================] - 1s 102us/step - loss: 0.3478
Epoch 6/10
13866/13866 [==============================] - 1s 103us/step - loss: 0.3478
Epoch 7/10
13866/13866 [==============================] - 1s 102us/step - loss: 0.3478
Epoch 8/10
13866/13866 [==============================] - 1s 102us/step - loss: 0.3477
Epoch 9/10
13866/13866 [==============================] - 1s 103us/step - loss: 0.3477
Epoch 10/10
13866/13866 [==============================] - 1s 103us/step - loss: 0.3477


In [118]:
model.save_weights('p100_lim_trained.h5')

In [119]:
from keras.models import load_model

new_model = model
for layer in new_model.layers[:3]:
    layer.trainable=False
for layer in new_model.layers[3:]:
    layer.trainable=True

model.compile(loss='mean_squared_error', optimizer='adam')


In [120]:
X = new_def_norm
Y = new_def_ipc_values
new_model.fit(X, Y, epochs=10, batch_size=10,  validation_split= 0.25, shuffle=True)

Train on 1938 samples, validate on 646 samples
Epoch 1/10
1938/1938 [==============================] - 1s 304us/step - loss: 0.1024 - val_loss: 0.0604
Epoch 2/10
1938/1938 [==============================] - 0s 94us/step - loss: 0.0789 - val_loss: 0.0469
Epoch 3/10
1938/1938 [==============================] - 0s 94us/step - loss: 0.0641 - val_loss: 0.0463
Epoch 4/10
1938/1938 [==============================] - 0s 92us/step - loss: 0.0614 - val_loss: 0.0463
Epoch 5/10
1938/1938 [==============================] - 0s 91us/step - loss: 0.0612 - val_loss: 0.0454
Epoch 6/10
1938/1938 [==============================] - 0s 93us/step - loss: 0.0614 - val_loss: 0.0492
Epoch 7/10
1938/1938 [==============================] - 0s 93us/step - loss: 0.0609 - val_loss: 0.0453
Epoch 8/10
1938/1938 [==============================] - 0s 93us/step - loss: 0.0609 - val_loss: 0.0453
Epoch 9/10
1938/1938 [==============================] - 0s 94us/step - loss: 0.0608 - val_loss: 0.0468
Epoch 10/10
1938/1938 [==